In [0]:
from pyspark.sql.functions import col, cast
from pyspark.sql.types import *

# Event Hub details
EH_CONN_STR= "Endpoint=sb://eventhub-namespace-twitter-analysis.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=Z5qEVHu9ZpCWVBM9zPFTFOLSAPPcRi1+6+AEhKq/Ggw="
EH_NAMESPACE = "eventhub-namespace-twitter-analysis" # my-event-hubs-namespace
EH_KAFKA_TOPIC = "eventhub-twitter-analysis" # my-event-hub
EH_BOOTSTRAP_SERVERS = f"{EH_NAMESPACE}.servicebus.windows.net:9093"
EH_SASL_WRITE = f"kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"$ConnectionString\" password=\"{EH_CONN_STR}\";"

# standard configuration options
topic_name = EH_KAFKA_TOPIC
eh_namespace_name = EH_NAMESPACE
eh_sasl = EH_SASL_WRITE
bootstrap_servers = EH_BOOTSTRAP_SERVERS
kafka_options = {
"kafka.bootstrap.servers": bootstrap_servers,
"kafka.sasl.mechanism": "PLAIN",
"kafka.security.protocol": "SASL_SSL",
"kafka.request.timeout.ms": "60000",
"kafka.session.timeout.ms": "30000",
"startingOffsets": "earliest",
"kafka.sasl.jaas.config": eh_sasl,
"subscribe": topic_name,
}


In [0]:
kafka_df = spark\
.readStream\
.format("kafka")\
.options(**kafka_options)\
.load()

In [0]:
query = kafka_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start() \

query.awaitTermination()